In [1]:
from icecream import ic
from yfiles_jupyter_graphs_for_kuzu import KuzuGraphWidget
import kuzu
import watermark

In [2]:
%load_ext watermark
%watermark
%watermark --iversions

Last updated: 2025-07-28T19:41:58.368921-07:00

Python implementation: CPython
Python version       : 3.13.3
IPython version      : 9.1.0

Compiler    : Clang 16.0.0 (clang-1600.0.26.6)
OS          : Darwin
Release     : 24.5.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

watermark                     : 2.5.0
kuzu                          : 0.9.0
yfiles_jupyter_graphs_for_kuzu: 0.0.4



## Extract one fraud network

In [4]:
DB_PATH = "./db"

db = kuzu.Database(DB_PATH)
conn = kuzu.Connection(db)

Create a yFiles graph widget so we can explore our graph interactively

In [5]:
g = KuzuGraphWidget(conn)

In [9]:
g.show_cypher(
    """
    MATCH (a)-[b]->(c:Entity)-[d *1..4]->(e)
    WHERE c.id = "sz_100036"
    RETURN * LIMIT 200;
    """,
    layout="circular"
)

GraphWidget(layout=Layout(height='710px', width='100%'))

Now let's extract the shell companies in this particular fraud network

In [24]:
res = conn.execute(
    """
    MATCH (a:Entity)-[b *1..3]->(c)
    WHERE a.descrip CONTAINS "Abassin"
      AND c.kind = "ORGANIZATION"
    RETURN c.name, COLLECT(DISTINCT c.addr)
    LIMIT 100;
    """
)

shells: dict = {
    row[0]: row[1]
    for row in res.get_as_pl().iter_rows()
}

ic(shells);

ic| shells: {'BARLLOWS SERVICES LTD': ['31 Quernmore Close, Bromley, Kent, United Kingdom, '
                                       'BR1 4EL',
                                       '3 Market Parade, 41 East Street, Bromley, BR1 1QN'],
             'LMAR (GB) LTD': ['31 Quernmore Close, Bromley, Kent, United Kingdom, BR1 '
                               '4EL'],
             'WELLHANCIA HEALTH CARE LTD': ['31 Quernmore Close, Bromley, BR1 4EL']}
